<a href="https://colab.research.google.com/github/ferjorosa/learn-pytorch/blob/main/FastAI%20NLP%20Course/Translation%2C%20attention%20and%20transformers/translation_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Translation with a Sequence to Sequence Network and Attention

In this project we will be teaching a neural network to translate from French to English.

```
[KEY: > input, = target, < output]

> il est en train de peindre un tableau .
= he is painting a picture .
< he is painting a picture .

> pourquoi ne pas essayer ce vin delicieux ?
= why not try that delicious wine ?
< why not try that delicious wine ?

> elle n est pas poete mais romanciere .
= she is not a poet but a novelist .
< she not not a poet but a novelist .

> vous etes trop maigre .
= you re too skinny .
< you re all alone .
```

This is made possible by the simple but powerful idea of the <a href="https://arxiv.org/abs/1409.3215">sequence to sequence network</a>, in which two recurrent neural networks work together to transform one sequence to another. An encoder network condenses an input sequence into a vector, and a decoder network unfolds that vector into a new sequence.

<img src="https://github.com/ferjorosa/learn-pytorch/blob/main/FastAI%20NLP%20Course/Translation%2C%20attention%20and%20transformers/images/sequence_to_sequence_example.png?raw=1">

To improve upon this model we’ll use an <a href="https://arxiv.org/abs/1409.0473">attention mechanism</a>, which lets the decoder learn to focus over a specific range of the input sequence.

In [1]:
# Only on Google Colab

# !rm -r data_translation

URL = "https://raw.githubusercontent.com/ferjorosa/learn-pytorch/main/FastAI%20NLP%20Course/Translation%2C%20attention%20and%20transformers/data_translation"

!wget -r --no-parent --cut-dirs=7 {URL}/eng-fra.txt

!mv raw.githubusercontent.com data_translation

--2022-04-01 11:06:24--  https://raw.githubusercontent.com/ferjorosa/learn-pytorch/main/FastAI%20NLP%20Course/Translation%2C%20attention%20and%20transformers/data_translation/eng-fra.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9541158 (9.1M) [text/plain]
Saving to: ‘raw.githubusercontent.com/eng-fra.txt’

raw.githubuserconte 100%[===================>]   9.10M  --.-KB/s    in 0.06s   

2022-04-01 11:06:24 (149 MB/s) - ‘raw.githubusercontent.com/eng-fra.txt’ saved [9541158/9541158]

FINISHED --2022-04-01 11:06:24--
Total wall clock time: 0.09s
Downloaded: 1 files, 9.1M in 0.06s (149 MB/s)


In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data

The data for this project consists of many thousands of English to French translation pairs. Data comes from an open-source website called <a href="https://tatoeba.org/">Tatoeba.org</a>, which has <a href="https://tatoeba.org/eng/downloads">downloads available</a>, and better yet, someone did the extra work of splitting language pairs into individual text files <a href="https://www.manythings.org/anki/">here</a>.

Similar to the character encoding used in the character-level RNN tutorials, we will be representing each word in a language as one-hot vector. Compared to the dozens of characters that might exist in a language, there are many many more words. We will however cheat a bit and trim the data to only use a few thousand words per language.

<img src="images/word_encoding.png">

We will need a unique index per word to use as the inputs and targets of the networks later. To keep track of all this we will use a helper class called `Lang` which has word -> index (`word2index`) and index -> word (`index2word`) dictionaries, as well as a count ofeach word `word2count` which will be used to replace rare words later.



In [3]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Data file is in Unicode. To simplify we will turn Unicode characters to ASCII, make everything lowercase, and trim most punctuation.

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

To read the data we will split the file into lines, and then split lines into pairs. 

The following function can be applied to all of the data files from [the Tatoeba-Anki download page](https://www.manythings.org/anki/).The files are generally English -> Other language. So, if we want to translate from Other Language -> English, we can set the `reverse` argument to `True`.

In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data_translation/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Since there are a lot of example sentences and we want to train something quickly, we’ll trim the data set to only relatively short and simple sentences. Here the maximum length is 10 words (that includes ending punctuation) and we’re filtering to sentences that translate to the form “I am” or “He is” etc. (accounting for apostrophes replaced earlier).

In [6]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

* Read text file and split into lines, split lines into pairs
* Normalize text, filter by length and content
* Make word lists from sentences in pairs


In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['je suis calme .', 'i m calm .']


## The Seq2Seq model

A recurrent neural network (RNN) is a network that operates on a sequence and uses its own output as inputfor subsequent steps.

A Sequence to Sequence network, or seq2seq network, or Encoder Decoder network, is a model consisting of two RNNs called the encoder and decoder. The encoder reads an input sequence and outputs a single vector, and the decoder reads that vector to produce an output sequence.

<img src="https://github.com/ferjorosa/learn-pytorch/blob/main/FastAI%20NLP%20Course/Translation%2C%20attention%20and%20transformers/images/sequence_to_sequence_example.png?raw=1">

Unlike sequence prediction with a single RNN, where every input corresponds to an output, the seq2seq model frees us from sequence length and order, which makes it ideal for translation between two languages.

Consider the sentence "Je ne suis pas le chat noir" -> "I am not the black cat". Most of the words in the input sentence have a direct translation in the output sentence, but are in slightly different orders, e.g. "chat noir" and "black cat". Because of the "ne pas" construction, there is also one more owrd in the input sequence. It would be difficult to produce a correct translation directly from the sequence of input words.

With a seq2seq model, the encoder creates a single vector which, in the ideal case, encodes the "meaning" of the input sequence (i.e., a single point in some N dimensional space of sentences).



### The encoder
The encoder of a seq2seq network isa RNN that outputs some value for every word from the input sentene. For every input word the encoder ouptus a vector and a hidden state, and uses the hidden state for the next input word.

<img src="images/encoder_diagram.png">

In this case we are considering a Gated recurrent unit (GRU) as the RNN of the encoder. Other RNN architectures such as LSTM could also be used. 

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### The decoder

The decoder is another RNN that takes the encoder output vector(s) and outputs a sequence of words to create the translation.

#### Simple decoder

In the simplest seq2seq decoder we only use the last output of the encoder. This last output is sometimes called the **context vector** as it encodes context from the entire sequence. This context vector is used as the initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and hidden state. The initial input token is the start of string `<SOS>` and the first hidden state is the context vector (the encoder's last hidden state).

<img src="images/simple_decoder_diagram.png">

In [9]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

#### Attention decoder

If only the context vector is passed between the encoder and the decoder, that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to focus on a different part of the encoder's ouputs for every step of the decoder's own outputs. First we calculate a set of **attention weights**. These will be multiplied by the encoder output vectors to create a weighted combination. The result (called `attn_applied` in the code) should contain information about that specific part of the input sequence, and thus help the decoderchoosethe right output words.

<img src="https://github.com/ferjorosa/learn-pytorch/blob/main/FastAI%20NLP%20Course/Translation%2C%20attention%20and%20transformers/images/attention_seq2seq.png?raw=1">

Calculating the attentionweights is done with a another feed-forward layer `attn`, using the decoder's input and hidden state as inputs. Because there are sentences of all sizes in the training data, to actually create and train this layer **we have to choose a maximum sentence length** (input length, for encoder outputs) that it can apply to. sentences of the maximum length will use all the attention weights, while shorter sentences will only use the first few.

<img src="https://github.com/ferjorosa/learn-pytorch/blob/main/FastAI%20NLP%20Course/Translation%2C%20attention%20and%20transformers/images/attention_decoder_diagram.png?raw=1">

In [10]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)